<a href="https://colab.research.google.com/github/kodenshacho/sigma/blob/master/glfragment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#include <GL/glut.h>

const char *fragmentProgram =
    "!!ARBfp1.0\n"
    "TEMP texture1, texture3;\n"
    "TEX texture1, fragment.texcoord[0], texture[1], 2D;\n"
    "TEX texture3, fragment.texcoord[0], texture[3], 2D;\n"
    "MUL result.color, texture1, {1.0, 1.0, 1.0, 1.0};\n" // White color for texture 1
    "MUL result.color, texture3, {1.0, 0.5, 0.0, 1.0};\n" // Orange color for texture 3
    "END\n";

void display() {
    glClear(GL_COLOR_BUFFER_BIT);

    glBegin(GL_QUADS);
    glTexCoord2f(0.0, 0.0); glVertex2f(-1.0, -1.0);
    glTexCoord2f(1.0, 0.0); glVertex2f(1.0, -1.0);
    glTexCoord2f(1.0, 1.0); glVertex2f(1.0, 1.0);
    glTexCoord2f(0.0, 1.0); glVertex2f(-1.0, 1.0);
    glEnd();

    glutSwapBuffers();
}

void init() {
    glEnable(GL_TEXTURE_2D);

    GLuint fragmentProgramId;
    glGenProgramsARB(1, &fragmentProgramId);
    glBindProgramARB(GL_FRAGMENT_PROGRAM_ARB, fragmentProgramId);
    glProgramStringARB(GL_FRAGMENT_PROGRAM_ARB, GL_PROGRAM_FORMAT_ASCII_ARB, strlen(fragmentProgram), fragmentProgram);

    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR);
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR);

    // Load textures and bind them

    glutDisplayFunc(display);
}

int main(int argc, char **argv) {
    glutInit(&argc, argv);
    glutInitDisplayMode(GLUT_DOUBLE | GLUT_RGBA);
    glutInitWindowSize(800, 600);
    glutCreateWindow("OpenGL ARB_fragment_program Example");
    init();
    glutMainLoop();
    return 0;
}

In [ ]:

!!ARBfp1.0

PARAM cyan = {0.0, 1.0, 1.0, 1.0};
PARAM orange = {1.0, 0.5, 0.0, 1.0};
TEMP textureColor1; //
TEMP textureColor3; //
TEMP combinedColor; //

//
TEX textureColor1, fragment.texcoord[0], texture[1], 2D;
//
MUL textureColor1, textureColor1, cyan;

//
TEX textureColor3, fragment.texcoord[0], texture[3], 2D;
//
MUL textureColor3, textureColor3, orange;

//
ADD combinedColor, textureColor1, textureColor3;

MOV result.color, combinedColor;

END

In [ ]:

!!ARBfp1.0
OPTION ARB_precision_hint_fastest;

PARAM showTextureParams = program.local[0];

TEMP texColor;

TEX texColor, fragment.texCoord[0], texture[0], 2D;

// 1
MUL_SAT texColor, texColor, showTextureParams.x;

// 2
TEX tmpTexColor, fragment.texCoord[0], texture[1], 2D;
MUL tmpTexColor, tmpTexColor, showTextureParams.y;
ADD_SAT texColor, texColor, tmpTexColor;

MUL result.color, fragment.color, texColor;
END

In [ ]:

#include <GL/glew.h>

#include <GLFW/glfw3.h>

#define NUM_COLORS 6

#define STACK_HEIGHT 6

#define FLAG_OFF 0

#define FLAG_ON 1

static const unsigned char imageData[NUM_COLORS][WIDTH][HEIGHT][4] = {



};

static const char* vertexShaderSource = R"(

    #version 330 core

    layout (location = 0) in vec3 aPos;

    layout (location = 1) in vec2 aTexCoord;

    out vec2 vTexCoord;

    void main() {

        gl_Position = vec4(aPos, 1.0);

        vTexCoord = aTexCoord;

    }

)";

static const char* fragmentShaderSource = R"(

    #version 330 core

    uniform sampler2D textures[NUM_COLORS];

    uniform int currentColorIndex;

    uniform int flag;

    in vec2 vTexCoord;

    out vec4 FragColor;

    void main() {

        if (currentColorIndex >= 0 && currentColorIndex < NUM_COLORS) {

            if (flag == FLAG_ON) {

                FragColor = texture(textures[currentColorIndex], vTexCoord);

            } else {

                FragColor = vec4(0.0);

            }

        } else {

            FragColor = vec4(0.0);

        }

    }

)";

int main() {

    if (!glfwInit()) {

        return -1;

    }

ㅣ glfwWindowHint(GLFW_CONTEXT_VERSION_MAJOR, 3);

    glfwWindowHint(GLFW_CONTEXT_VERSION_MINOR, 3);

    glfwWindowHint(GLFW_OPENGL_PROFILE, GLFW_OPENGL_CORE_PROFILE);

    GLFWwindow* window = glfwCreateWindow(WIDTH, HEIGHT, "OpenGL Image Stacking", NULL, NULL);

    if (!window) {

        glfwTerminate();

        return -1;

    }

    glfwMakeCurrentContext(window);

    glfwSetFramebufferSizeCallback(window, framebufferSizeCallback);

    glewExperimental = GL_TRUE;

    if (glewInit() != GLEW_OK) {

        glfwTerminate();

        return -1;

    }

    GLuint textures[NUM_COLORS];

    glGenTextures(NUM_COLORS, textures);

    for (int i = 0; i < NUM_COLORS; ++i) {

        glBindTexture(GL_TEXTURE_2D, textures[i]);

        glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, WIDTH, HEIGHT, 0, GL_RGBA, GL_UNSIGNED_BYTE, imageData[i]);

        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR);

        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR);

        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT);

        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT);

    }

    GLuint vertexShader = glCreateShader(GL_VERTEX_SHADER);

    glShaderSource(vertexShader, 1, &vertexShaderSource, NULL);

    glCompileShader(vertexShader);

    GLuint fragmentShader = glCreateShader(GL_FRAGMENT_SHADER);

    glShaderSource(fragmentShader, 1, &fragmentShaderSource, NULL);

    glCompileShader(fragmentShader);

    GLuint shaderProgram = glCreateProgram();

    glAttachShader(shaderProgram, vertexShader);

    glAttachShader(shaderProgram, fragmentShader);

    glLinkProgram(shaderProgram);

    //